# <center>An Introduction to Monte Carlo Methods</center>
Monte Carlo methods are statistical techniques that solves complex mathematical problems. Its name refers to the Monte Carlo Casino in Monaco. The method is first proposed by Stanislaw Ulam in 1940s. He became interested in plotting the outcomes of solitare games and mapping the distribution of it during recovery from an illness. Later Jon Von Neumann collaborated with Ulam to develop Monte Carlo methods. The core idea behind the method is fairly simple and easy to implement. Yet, it can be used to solve very complex problems. Monte Carlo method is often the easiest way to approach a mathematical problem. Sometimes it may even be the only solution. 

Monte Carlo is not a single algorithm but rather a family of problem solving techniques. All of which uses random numbers to build a probabilistic model that approximates the optimum solution. These techniques include Monte Carlo methods, Monte Carlo simulations, Monte Carlo tree search, etc. Monte Carlo tree search specifically, is useful for searching the best move in the game. AlphaGo is one of the famous implementations of Monte Carlo tree search.

In this notebook, we will introduce some of the applications of Monte Carlo methods.

In [1]:
import matplotlib
import numpy as np
import pandas as pd
import random
from ipywidgets import widgets
from bokeh.io import output_notebook, push_notebook
from bokeh.plotting import figure, ColumnDataSource, show, gridplot
from bokeh.models import CustomJS,Select

output_notebook()

def random_coordinates(n):
    """
    Initialize random x and y coordinates between 0 and 1
    
    Arguments:
        n(int): number of coordinates
    Returns:
        pandas Dataframe containing x and y coordinates
    """
    x = [random.random() for i in range(n)]
    y = [random.random() for i in range(n)]
    return pd.DataFrame(list(zip(x,y)), columns = ['x','y'])

Loading BokehJS ...

## Estimating the Value of &pi;

One method to estimate the value of &pi; is by using a Monte Carlo method. The method can be desribed with the following: Draw a square and a quarter circle in a canvas. Then generate a large number of random points within the square and observe how many fall in the quarter circle.

In [2]:
#figure 1
df1 = random_coordinates(500)
fig1 = figure(width=400, height=400,match_aspect = True)
fig1.wedge(x=0,y=0,radius=1,start_angle = 0, end_angle = np.pi/2, color='PaleGreen',fill_alpha = 0.8)
fig1.circle(x='x',y='y',size=3,source=ColumnDataSource(df1),color='SkyBlue')
show(fig1)

- The area of the quater circle is $1/4\pi r^{2}$

- The area of the square is $r^{2}$

- The ratio between the area of the quarter circle and square is $1/4\pi$

- The same ratio shall apply between the number of points within the square (the total number of points)  and the number of points within the quarter circle. Therefore, $\pi$ can be estimated through the following equation:

 &emsp; $\pi \approx 4 \times$ (# of points within the circle / total # of points)

<br>
<br>
<br>
We are now able to estimate the value of &pi;:

In [3]:
# Calculate Pi
def estimate_pi(data):
    n = data.shape[0]
    coordinates = np.asarray(data)
    count = 0
    for i in range(n):
        x = coordinates[i][0]
        y = coordinates[i][1]
        if(x*x+y*y < 1):
            count += 1
    return (count/n)*4

print('Pi estimation using 10000 points: ',"{:.5f}".format(estimate_pi(random_coordinates(10000))))

Pi estimation using 10000 points:  3.14360


Lets now plot the points and see how the total number of points affect the estimation:

In [4]:
#figure2
n = 500
df2 = random_coordinates(n)
source2 = ColumnDataSource(df2)
fig2 = figure(width=400, height=400,match_aspect = True)
fig2.wedge(x=0,y=0,radius=1,start_angle = 0, end_angle = np.pi/2, color='PaleGreen',fill_alpha = 0.8)
fig2.circle(x='x',y='y',size=3,source=source2,color='SkyBlue')
handle1 = show(fig2, notebook_handle=True)
n_slider = widgets.IntSlider(
    description="# of Random Points(n)",
    min=500,
    max=5000,
    value=500,
    step=500,
)

def n_change_handler(change):
    n = change.new
    new_data = random_coordinates(n)
    source2 = ColumnDataSource(new_data)
    #clear renderers
    fig2.renderers = [fig2.renderers[0]]
    #add new renderer
    fig2.circle(x='x',y='y',size=3,source=source2,color='SkyBlue')
    push_notebook(handle=handle1)
    print('\rPi Estimation: ',"{:.5f}".format(estimate_pi(new_data)),end='')

n_slider.observe(n_change_handler, names='value')
display(n_slider)

IntSlider(value=500, description='# of Random Points(n)', max=5000, min=500, step=500)

Pi Estimation:  3.18400

To visualize how the total number of random points affect the value of pi estimation, lets plot these two values(this can take a few seconds):

In [5]:
#figure 3
n_points = [1000*(i+1) for i in range(50)]
pi_estimations = np.empty(shape=(len(n_points),2))
for i in range(len(n_points)):
    num = n_points[i]
    pi_estimations[i] = [num,estimate_pi(random_coordinates(num))]

fig3 = figure(width=600,height=300,x_range=(0,50000),y_range=(3.1,3.2),match_aspect=True)
fig3.circle(x=pi_estimations[:,0], y=pi_estimations[:,1] ,color='SkyBlue',size=5,legend_label='\u03C0 estimations')
fig3.line([0,n_points[-1]],np.pi,legend_label='\u03C0',color='PaleGreen',width=3)
show(fig3)

As more points are being used for the process, the estimations is getting closer to the actual value of &pi;

<br>
However, we only performed one trial for a specific number of random points. One way to optimize our answer will be taking the mean of multiple trials:

In [6]:
estimations = 0
for i in range(100):
    estimations += estimate_pi(random_coordinates(10000))
estimations /= 100
print('Pi estimation using 10000 points with 100 trials: ',"{:.5f}".format(estimations))

Pi estimation using 10000 points with 100 trials:  3.14044


## Estimating Integrals

One of the most famous applications of Monte Carlo Method is estimating an integral. 

### First approach
The process can be very similar to what we did for estimating &pi;.

We generate a large number of random points within the square and count how many points is in the area of the curve that we are estimating:

- The curve y = x is used in the demo for simplicity. 

In [7]:
#figure 4
n_int = 500
df3 = random_coordinates(n_int)
source3 = ColumnDataSource(df3)
fig4 = figure(width=400, height=400,match_aspect = True)
fig4.varea(x=[0,1],y1=[0,1],y2=[0,0],color='PaleGreen',legend_label='y=x',fill_alpha=0.8)
fig4.circle(x='x',y='y',size=3,source=source3,color='SkyBlue')
fig4.legend.location = 'top_left'
handle2 = show(fig4, notebook_handle=True)
n_int_slider = widgets.IntSlider(
    description="Number of Random Points(n)",
    min=500,
    max=5000,
    value=500,
    step=500,
)

def n_int_change_handler(change):
    n_int = change.new
    new_data = random_coordinates(n_int)
    source3 = ColumnDataSource(new_data)
    #clear renderers
    fig4.renderers = [fig4.renderers[0]]
    #add new renderer
    fig4.circle(x='x',y='y',size=3,source=source3,color='SkyBlue')
    new_data = np.asarray(new_data)
    count = 0
    for entry in new_data:
        if(entry[0] > entry[1]):
            count += 1
    area = count/n_int
    error = abs(1-(area/0.5))
    print('\rArea = ',"{:.5f}".format(area),',  Error: ',"{:.5f}".format(error), end='')
    push_notebook(handle=handle2)

n_int_slider.observe(n_int_change_handler, names='value')
display(n_int_slider)

IntSlider(value=500, description='Number of Random Points(n)', max=5000, min=500, step=500)

### Second Approach

Another approach to the problem is often used. Lets say we want to calculate the area under the curve $f(x)$. Instead of picking random points in the square, we can randomly pick within the domain of the curve (a,b). We can estimate the area under the curve by $f(x)*(b-a)$. By taking a large number of random x values, we will be able to estimate the integral by taking the average.

In [8]:
#figure 5
figures = []
for i in range(6):
    randnum = random.random()
    fig5 = figure(width=200, height=200,match_aspect = True)
    fig5.varea(x=[0,1],y1=[randnum,randnum],y2=[0,0],color='PaleGreen',fill_alpha=0.8)
    fig5.line(x=[0,1],y=[0,1],line_width=2)
    figures.append(fig5)

show(gridplot(figures,ncols=3))

Now we can estimate the integral:

In [9]:
integral = 0
for i in range(10000):
    #In this particular case,f(x)*(b-a) = x
    integral += random.random()
integral /= 10000
error = abs(1-(integral/0.5))
print('\rArea = ',"{:.5f}".format(integral),',  Error: ',"{:.5f}".format(error), end='')

Area =  0.49812 ,  Error:  0.00376

## Sales Forecast

Financial experts often use Monte Carlo methods to evaluate business models or evaluate risks and uncertainties. In this section, we will evaluate a business plan of bubble tea shop.

The net profit of a business can be calculated by:

&emsp; Net Profit = Sales Volume* (Selling Price - Unit cost) - Fixed costs

Lets say our bubble tea shop only sells one kind of drink, and the drink is sold at \\$8 in an okay year. You are able to sell 50,000 drinks this year, making \\$400,000 of revenue. 

In a bad year, customers dont drink as much bubble tea so you will only be able to sell 30,000 drinks. You need to price the drink higher at \\$11 though, because you don't want your revenue to look bad. You will make \\$330,000 revenue in a bad year.

In a good year, you will be able to sell 80,000 drinks. However, competitors see a big market in bubble tea and decided to join. You will have to lower your price at \\$6 to compete with other bubble tea shops. You will make \\$440,000 revenue in a good year.

Lets say there is an equal chance of being a bad, okay, or a good year.

The cost of the drink can be around \\$5 with standard deviation of \\$1 because of the fluctuation in the price of ingredients.

Additionally, your shop needs \\$140,000 each year for rent, salaries, etc.

We can now calculate how much we can make in a year:

In [10]:
# The number of sales, and selling price per unit according to the sentiment of the market
market = {'Bad':(30000,11),
         'Okay':(50000,8),
         'Good':(80000,6)}
fixed_cost = 140000
market_selector = widgets.Select(
    options=market.keys(),
    value='Okay',
    description='Market is:',
    disabled=False
)
#Unit cost can fluctuate around $5
#Net Profit = Sales Volume* (Selling Price - Unit cost) - Fixed costs
def market_change_handler(change):
    sentiment = change.new
    cost = np.random.normal(5,1)
    volume = market[sentiment][0]
    price = market[sentiment][1]
    profit = volume*(price - cost)-fixed_cost
    print('\rProfit in a random',sentiment.lower(),'year: ',"{:.2f}".format(profit),end='')

market_selector.observe(market_change_handler,names='value')
display(market_selector)

Select(description='Market is:', index=1, options=('Bad', 'Okay', 'Good'), value='Okay')

Profit in a random bad year:  21566.7845

But how do we know how much we are going to make in any year?

Intuitively, we can predict the sales by taking the average of the quantities to obtain the profit for a year.

* Sales Volume = (30000+50000+80000)/3 = 53333.33
* Selling Price = (11+8+6)/3 = 8.33
* Unit cost = 5

Therefore, our net profit for a year will be:

&emsp;Net Profit = 53333.33*(8.33-5) - 140000 = 37777.77

Great! We can make over \\$30000 a year!

<br>

However, predicting net profit with a Monte Carlo approach tells us a different answer:

In [11]:
n_trials_input = widgets.Text(
    value='0',
    description='N trials:',
    disabled=False
)

def calculate_profit(n_trials):
    n_trials = int(n_trials.value)
    if(n_trials != 0): 
        profit = 0
        for i in range(n_trials):
            sentiment = random.choice(list(market.keys()))
            cost = np.random.normal(5,1)
            volume = market[sentiment][0]
            price = market[sentiment][1]
            profit += volume*(price - cost)-fixed_cost
        print('\rProjected profit:',"{:.2f}".format(profit/n_trials),end='')

n_trials_input.on_submit(calculate_profit)
display(n_trials_input)

Text(value='0', description='N trials:')

Projected profit: -3671.94

As we increase N, we observe that the bubble tea shop is projected to not be profitable at all !!!

## Conclusion
Monte Carlo methods are powerful yet easy and flexible. Because of the increasing speed of modern computers, the execution of Monte Carlo methods do not consume a lot of time. However, Monte Carlo methods have their limitations. One of which being there are always unknown factors that Monte Carlo methods cannot account, especially in the area of finance. Also, we are unable to model correlations between the input and the output. Hence, we cannot learn anything from the process. 

## References
1. https://www.degruyter.com/document/doi/10.1515/mcma-2016-0102/html
2. https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html
3. https://www.solver.com/monte-carlo-simulation-example